# Wav2Vec

In [5]:
import torch
torch.cuda.is_available()
torch.cuda.device_count()

1

## Pretrained model example

In [2]:
import torch
import fairseq

cp_path = '/home/bayuan/Documents/ecog2txt_infonce/wav2vec_large.pt'#'/path/to/wav2vec.pt'
model, cfg, task = fairseq.checkpoint_utils.load_model_ensemble_and_task([cp_path])
model = model[0]
model.eval()

wav_input_16khz = torch.randn(1,10000)
z = model.feature_extractor(wav_input_16khz)
c = model.feature_aggregator(z)

OSError: Model file not found: /home/bayuan/Documents/ecog2txt_infonce/wav2vec_large.pt

In [2]:
print(c)

tensor([[[1.1368e-02, 1.1790e-02, 1.5433e-02,  ..., 2.7389e-02,
          3.0830e-02, 3.8877e-02],
         [9.4709e-03, 1.0406e-03, 2.3769e-02,  ..., 7.1879e-02,
          9.9600e-02, 1.3635e-01],
         [0.0000e+00, 0.0000e+00, 4.9162e-04,  ..., 3.2663e-04,
          0.0000e+00, 7.4486e-04],
         ...,
         [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 1.4083e-03,
          4.7111e-03, 7.9672e-03],
         [1.7154e-01, 1.9224e-01, 2.5076e-01,  ..., 0.0000e+00,
          2.0530e-02, 0.0000e+00],
         [3.7883e-01, 3.7031e-01, 3.8092e-01,  ..., 3.1227e-01,
          3.7051e-01, 3.6118e-01]]], grad_fn=<MulBackward0>)


## Training a new model

### Prepare training data manifest

In [44]:
import numpy as np
from scipy.io.wavfile import write

for i in range(30):
    wav_input_16khz = np.random.normal(0, 1, 10000)
    # print(wav_input_16khz.shape)
    write(f'/home/brian/Documents/ECE496/ecog2txt_infonce/testwavs/wav_{i}.wav', 16000, wav_input_16khz)

(10000,)
(10000,)
(10000,)
(10000,)
(10000,)
(10000,)
(10000,)
(10000,)
(10000,)
(10000,)
(10000,)
(10000,)
(10000,)
(10000,)
(10000,)
(10000,)
(10000,)
(10000,)
(10000,)
(10000,)
(10000,)
(10000,)
(10000,)
(10000,)
(10000,)
(10000,)
(10000,)
(10000,)
(10000,)
(10000,)


In [7]:
!python3 /home/bayuan/Documents/fall23/fairseq/examples/wav2vec/wav2vec_manifest.py \
    /home/bayuan/Documents/fall23/ecog2vec/testwavs \
  --dest /home/bayuan/Documents/fall23/ecog2vec/manifest \
  --ext wav \
  --valid-percent 0.05

### Train a wav2vec model


In [6]:
# !module load use.own
# !PYTHONPATH=/depot/jgmakin/data/conda_env/wav2vec/lib/python3.8/site-packages:/apps/cent7/xalt/site:/apps/cent7/xalt/libexec:/apps/spack/gilbreth/apps/intel-parallel-studio/cluster.2017.1-intel-17.0.1-2off4ih/advisor_2017.1.1.486553/pythonapi
# !echo $PYTHONPATH
# !module load conda-env/wav2vec-py3.8.5
!python3 -c 'import argparse; print(argparse.__file__)'
!python3 /home/bayuan/Documents/fall23/fairseq/train.py \
    /home/bayuan/Documents/fall23/ecog2vec/manifest \
  --save-dir /home/bayuan/Documents/fall23/ecog2vec/model \
  --num-workers 6 --fp16 --max-update 400000 --save-interval 1 --no-epoch-checkpoints \
  --arch wav2vec --task audio_pretraining --min-lr 1e-06 --stop-min-lr 1e-09 --optimizer adam --lr 0.001 --lr-scheduler cosine \
  --conv-feature-layers "[(512, 10, 5), (512, 8, 4), (512, 4, 2), (512, 4, 2), (512, 4, 2), (512, 1, 1), (512, 1, 1)]" \
  --conv-aggregator-layers "[(512, 2, 1), (512, 3, 1), (512, 4, 1), (512, 5, 1), (512, 6, 1), (512, 7, 1), (512, 8, 1), (512, 9, 1), (512, 10, 1), (512, 11, 1), (512, 12, 1), (512, 13, 1)]" \
  --skip-connections-agg --residual-scale 0.5 --log-compression --warmup-updates 500 --warmup-init-lr 1e-07 --criterion wav2vec --num-negatives 10 \
  --max-sample-size 150000 --skip-invalid-size-inputs-valid-test --max-epoch 20 --max-tokens 1500000 --batch-size 1

/home/bayuan/anaconda3/envs/wav2vec/lib/python3.9/argparse.py
2023-10-18 09:13:18 | INFO | fairseq_cli.train | {'_name': None, 'common': {'_name': None, 'no_progress_bar': False, 'log_interval': 100, 'log_format': None, 'log_file': None, 'aim_repo': None, 'aim_run_hash': None, 'tensorboard_logdir': None, 'wandb_project': None, 'azureml_logging': False, 'seed': 1, 'cpu': False, 'tpu': False, 'bf16': False, 'memory_efficient_bf16': False, 'fp16': True, 'memory_efficient_fp16': False, 'fp16_no_flatten_grads': False, 'fp16_init_scale': 128, 'fp16_scale_window': None, 'fp16_scale_tolerance': 0.0, 'on_cpu_convert_precision': False, 'min_loss_scale': 0.0001, 'threshold_loss_scale': None, 'amp': False, 'amp_batch_retries': 2, 'amp_init_scale': 128, 'amp_scale_window': None, 'user_dir': None, 'empty_cache_freq': 0, 'all_gather_list_size': 16384, 'model_parallel_size': 1, 'quantization_config_path': None, 'profile': False, 'reset_logging': False, 'suppress_crashes': False, 'use_plasma_view': Fal

## Extract embeddings from the downstream task data


In [ ]:
!PYTHONPATH=/path/to/fairseq python examples/wav2vec/wav2vec_featurize.py --input /path/to/task/waves --output /path/to/output \
  --model /home/bayuan/Documents/ecog2txt_infonce/model/checkpoint_best.pt --split train valid test